In [ ]:
import sys
import numpy as np
import scipy.misc
import os
import tensorflow as tf
import ops

In [ ]:
from glob import glob
data = glob(os.path.join("/home/zhangjinbin/research/data/", "*.jpg"))

In [ ]:
slim = tf.contrib.slim

In [ ]:
def _conv_layer(net,filter_size,strides,num_filters, scope,x_att=None):
    with tf.variable_scope(scope):
        if x_att==None:
            weights_init = _conv_init_vars(net, num_filters, filter_size,scope)
            strides_shape = [1, strides, strides, 1]
            padding = filter_size // 2
            padded_input = tf.pad(
                net, [[0, 0], [padding, padding], [padding, padding], [0, 0]],mode='REFLECT')
            net = tf.nn.conv2d(padded_input, weights_init, strides_shape, padding='VALID')
            return net
        else:
            weights_init = _conv_init_vars(net, num_filters, filter_size,scope)
            #[filter_size, filter_size, in_channels, out_channels]
            filter_size, filter_size, in_channels, out_channels = [i.value for i in weights_init.get_shape()]
            hidden  = tf.nn.relu(linear(x_att,  1000, scope='c_h1', stddev=0.01))
            control_net = tf.nn.sigmoid(linear(hidden,filter_size*filter_size*in_channels*out_channels,scope='c_wc',stddev=0.01))
            control_net= tf.reshape(control_net,[-1,filter_size, filter_size, in_channels, out_channels])
            weights_initnew=[]
            for i in range(batch_size):
                weights_initi = tf.mul(control_net[i,:,:,:],weights_init)
                weights_initnew.append(weights_initi)
            #scale_c = tf.nn.sigmoid(linear(hidden,  rows*cols*channels, scope='c_sc', stddev=0.01))
            strides_shape = [1, strides, strides, 1]
            padding = filter_size // 2
            padded_input = tf.pad(
                net, [[0, 0], [padding, padding], [padding, padding], [0, 0]],mode='REFLECT')
            conv_out= []
            for i in range(batch_size):
                conv_out.append(tf.nn.conv2d([padded_input[i,:,:,:]], weights_initnew[i], strides_shape, padding='VALID'))   
            #net = _instance_norm(net)
            net = tf.concat(0,conv_out)
            return net
def _conv_init_vars(net, out_channels, filter_size,scope, transpose=False,stddev =0.01):
    _, rows, cols, in_channels = [i.value for i in net.get_shape()]
    if not transpose:
        weights_shape = [filter_size, filter_size, in_channels, out_channels]
    else:
        weights_shape = [filter_size, filter_size, out_channels, in_channels]
    norm = tf.random_normal_initializer(stddev=stddev)
    with tf.variable_scope(scope):
        weights_init = tf.get_variable('weight',weights_shape, initializer=norm)
    return weights_init
def _residual_block(input_, kernel_size, scope,x_att=None):
    with tf.variable_scope(scope):
        num_outputs = input_.get_shape()[-1].value
        h_1 = _conv_layer(input_, kernel_size, 1, num_outputs, 'conv1',x_att=x_att)
        h_2 = _conv_layer(h_1, kernel_size, 1, num_outputs, 'conv2',x_att=x_att)
        return input_ + h_2
def _upsampling(input_, kernel_size, stride, num_outputs, scope):
    if kernel_size % 2 == 0:
        raise ValueError('kernel_size is expected to be odd.')
    with tf.variable_scope(scope):
        _, height, width, _ = [s.value for s in input_.get_shape()]
        upsampled_input = tf.image.resize_nearest_neighbor(
        input_, [stride * height, stride * width])
        return _conv_layer(upsampled_input, kernel_size, 1, num_outputs, 'conv')

def _instance_norm(net,x_att,batch_size,scope,train=True):
    batch, rows, cols, channels = [i.value for i in net.get_shape()]
    var_shape = [channels]
    mu, sigma_sq = tf.nn.moments(net, [1,2], keep_dims=True)
    shift = tf.Variable(tf.zeros(var_shape))
    scale = tf.Variable(tf.ones(var_shape))
    epsilon = 1e-3
    normalized = (net-mu)/(sigma_sq + epsilon)**(.5)
    return scale * normalized + shift
def shift_scale_get(x_att,net,scope):
    norm = tf.random_normal_initializer(stddev=0.001)
    const = tf.constant_initializer(1.0)
    batch, rows, cols, channels = [i.value for i in net.get_shape()]
    with tf.variable_scope(scope):
        hidden  = tf.nn.relu(linear(x_att,  1000, scope='c_h1', stddev=0.01))
        shift_c = tf.nn.sigmoid(linear(hidden,  rows*cols*channels, scope='c_sh', stddev=0.01))
        scale_c = tf.nn.sigmoid(linear(hidden,  rows*cols*channels, scope='c_sc', stddev=0.01))
        shift = tf.get_variable('shift', [batch_size,channels], initializer=norm)
        scale = tf.get_variable('scalse', [batch_size,channels], initializer=const)
        shift_ = tf.reshape(shift_c,[-1,rows, cols, channels])#tf.mul(shift,shift_c)+tf.mul(shift,tf.sub(tf.ones_like(shift_c),shift_c))
        scale_ = tf.reshape(scale_c,[-1,rows, cols, channels])#tf.mul(scale,scale_c)+tf.mul(scale,tf.sub(tf.ones_like(scale_c),scale_c))
        print('mormshift:')
        print(shift_.get_shape())
    return shift_,scale_
def linear(input, output_dim, scope=None, stddev=0.01):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope or 'linear'):
        w = tf.get_variable('w', [input.get_shape()[1], output_dim], initializer=norm)
        b = tf.get_variable('b', [output_dim], initializer=const)
        return tf.matmul(input, w) + b

In [ ]:
def recognition_network(images,reuse = False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.variable_scope('r_recognition', reuse=reuse):
        with tf.variable_scope('r_dis'):
            h = _conv_layer(images, 9, 2, 32, 'r_conv1')
            h = tf.nn.relu(h)
            h = _conv_layer(h, 3, 2, 64, 'r_conv2')
            h = tf.nn.relu(h)
            h = _conv_layer(h, 3, 2, 128, 'r_conv3')
            h = tf.nn.relu(h)
            h = _conv_layer(h, 3, 2, 256, 'r_conv4')
            h = tf.nn.relu(h)
            h = _conv_layer(h, 3, 2, 512, 'r_conv5')
            h = tf.nn.relu(h)
            h = _conv_layer(h, 3, 2, 1024, 'r_conv6')
            h = tf.reshape(h,[-1,1024])
            h = tf.nn.relu(h)
            print(h.get_shape())
            output = tf.nn.relu(linear(h, 1000, scope='r_layer1', stddev=0.01))
            output = tf.nn.softmax(linear(output, 2, scope='r_layer2', stddev=0.01))
            return output

In [ ]:
def transform(input_,x_att,batch_size,reuse=False):
    with tf.variable_scope('c_transformer', reuse=reuse):
        with tf.variable_scope('c_contract'):
            h = _conv_layer(input_, 9, 1, 32, 'c_conv1',x_att=x_att )
            h = _instance_norm(h,x_att,batch_size,'c_norm1',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _conv_layer(h, 3, 2, 64, 'c_conv2',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm2',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _conv_layer(h, 3, 2, 128, 'c_conv3',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm3',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
        with tf.variable_scope('c_residual'):
            h = _residual_block(h, 3, 'c_residual1',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm4',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _residual_block(h, 3, 'c_residual2',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm5',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _residual_block(h, 3, 'c_residual3',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm6',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _residual_block(h, 3, 'c_residual4',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm7',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _residual_block(h, 3, 'c_residual5',x_att=x_att)
            h = _instance_norm(h,x_att,batch_size,'c_norm8',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
        with tf.variable_scope('c_expand'):
            h = _upsampling(h, 3, 2, 64, 'c_conv1_')
            h = _instance_norm(h,x_att,batch_size,'c_norm9',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            h = _upsampling(h, 3, 2, 32, 'c_conv2_')
            h = _instance_norm(h,x_att,batch_size,'c_norm10',train=True)
            h = tf.nn.relu(h)
            print(h.get_shape())
            return _upsampling(h, 9, 1, 3, 'c_conv3_')

In [ ]:
batch_size = 32
# weights_shape=[4,4,3,12]
# weights_init = tf.Variable(tf.truncated_normal(weights_shape, stddev=0.01), dtype=tf.float32)
input_img = tf.placeholder(tf.float32, shape=[batch_size,64,64,3], name="input_img")
x_attr = tf.placeholder(tf.float32, [batch_size, 2], name="input_attri")


out_img = transform(input_img/255.0,x_attr,batch_size)
reconstruction_loss = tf.reduce_mean(tf.squared_difference(out_img,input_img/255.0))
pre_att_input = recognition_network(input_img/255.0)
y_pre =  tf.argmax(pre_att_input, 1, name="predictions")
x_attr_ = tf.argmax(x_attr, 1, name="true_predictions")
correct_predictions = tf.equal(x_attr_, y_pre)
accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

pre_att_gen = recognition_network(out_img,reuse =True)
recog_attr_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pre_att_input,x_attr))
gen_attr_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pre_att_gen,x_attr))

In [ ]:
loss = reconstruction_loss + 10*gen_attr_loss
t_vars = tf.trainable_variables()
contr_params = [var for var in t_vars if 'c_' in var.name]
recog_params = [var for var in t_vars if 'r_' in var.name]

optim = tf.train.AdamOptimizer(learning_rate=1e-3). \
        minimize(loss,var_list=contr_params) 
optim_re_attr= tf.train.AdamOptimizer(learning_rate=1e-3). \
        minimize(recog_attr_loss,var_list=recog_params) 


In [ ]:
def get_imgs(path_batch):
    batch_img = []
    for path in path_batch:
        img = scipy.misc.imread(path, mode='RGB')
        batch_img.append(img)
    return batch_img

In [ ]:
import numpy as np
def onetozero(x):
    if x ==-1:
        return 0
    else:
        return x
attributes ={}
#[4,5,8,9,15,20,26,31]
att_all = []
with open('list_attr_celeba.txt', 'r') as f:
    f.readline()
    attribute_names = f.readline().strip().split(' ')
    for i, line in enumerate(f):
        fields = line.strip().replace('  ', ' ').split(' ')
        img_name = fields[0]
        if int(img_name[:6]) != i + 1:
            raise ValueError('Parse error.')
        attr_vec = map(int, fields[1:])
        att = attr_vec[15]
        if att ==-1:
            att =  0
        att_hot = [0,0]
        att_hot[att]=1
        attributes[img_name]= att_hot
        att_all.append(att_hot)

In [ ]:
os.environ.setdefault('CUDA_VISIBLE_DEVICES','1')
session_conf = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7),
                              allow_soft_placement=True,
                              log_device_placement=False)
sess = tf.InteractiveSession(config=session_conf)
# Train
tf.initialize_all_variables().run()
step =0
# data_batch = data[step*batch_size:(step+1)*batch_size]
# x_batch = get_imgs(data_batch)
# attr_batch = [attributes[element[-10:]] for element in data_batch]
# ot1,loss, recon_image,_ = sess.run([re_att,reconstruction_loss,out_img,optim_recog],
#                                feed_dict={input_img:x_batch ,x_attr:attr_batch})
#for epoch in range(epoches):
for step in range(2001):
    data_batch = data[step*batch_size:(step+1)*batch_size]
    x_batch = get_imgs(data_batch)
    attr_batch = [attributes[element[-10:]] for element in data_batch]
    acc,loss_,reloss,recon_image,_,_= sess.run([accuracy,loss,reconstruction_loss,out_img,optim_re_attr,optim],
                                   feed_dict={input_img:x_batch ,x_attr:attr_batch})
    
    if step %100 ==0:
        scipy.misc.imsave('./sample/%s_input.png'% (step),x_batch[0])
        scipy.misc.imsave('./sample/%s_output.png'% (step),recon_image[0]) 
    if step%20 ==0:
        print('step :'+str(step)+' loss :'+str(loss_)+' re_loss :'+str(reloss)+' acc :'+str(acc))

In [ ]:
att_all[-8:]

In [ ]:
#batch_xs, batch_ys = mnist.test.next_batch(30)
#Bald Bangs Black_Hair Blond_Hair Eyeglasses Male Pale_Skin Smiling
test_y = [[0, 1],
          [0, 1],
          [1, 0],
          [1, 0],
          [0, 1],
          [0, 1],
          [1, 0],
          [0, 1]]
tail_test = att_all[-24:]
test_y.extend(tail_test)

In [ ]:
data_batch = data[-32:]
test_batch = get_imgs(data_batch)
test_image = []
for i in range(32):
    feed_dict={input_img:test_batch,x_attr:np.repeat([test_y[i]],32, axis=0)}
    samples_test= sess.run(out_img, feed_dict=feed_dict)
    test_image.append(samples_test)
test_image = np.array(test_image) 

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def merge(images):
    h, w = 64,64
    img = np.zeros((h * 8, w * 8, 3))
    for j in range(8):
        for i in range(8):
            img[j*h:j*h+h, i*w:i*w+w, :] = images[i,j,:,:,:]
    return img

In [ ]:

#img_org = reshape_image(test_batch,8,1)

In [ ]:
img_test = merge(test_image)
f,axarr=plt.subplots(1,1,figsize=(6,6),sharey=True)
# samples
axarr.imshow(img_test,interpolation='none')
axarr.set_title('original image')

In [ ]:
# # Create the model
# x = tf.placeholder(tf.float32, [None, 784])
# y_ = tf.placeholder(tf.float32, [None, 10])

# hidden  = tf.nn.relu(linear(y_,  1000, scope='c_h1', stddev=0.01))
# a1 = tf.nn.relu(linear(hidden,  1000, scope='c_a1', stddev=0.01))
# a2 = tf.nn.relu(linear(hidden,  1000, scope='c_a2', stddev=0.01))
# a3 = tf.nn.relu(linear(hidden,  1000, scope='c_a3', stddev=0.01))
# b1 = tf.nn.relu(linear(hidden,  1000, scope='c_b1', stddev=0.01))
# b2 = tf.nn.relu(linear(hidden,  1000, scope='c_b2', stddev=0.01))
# b3 = tf.nn.relu(linear(hidden,  1000, scope='c_b3', stddev=0.01))



# h1 =linear(x,  1000, scope='c_layer1', stddev=0.01)
# h1_c = tf.nn.relu(tf.mul(h1,a1)+b1)
# h2 = tf.nn.relu(linear(h1_c, 1000, scope='c_layer2', stddev=0.01))
# h2_c = tf.nn.relu(tf.mul(h2,a2)+b2)
# h3 = tf.nn.relu(linear(h2_c, 1000, scope='c_layer3', stddev=0.01))
# h3_c = tf.nn.relu(tf.mul(h3,a3)+b3)
# h4 = linear(h3_c, 784, scope='c_layer4', stddev=0.01)
# out_vecter = tf.sigmoid(h4)
# out_image = tf.reshape(out_vecter,[-1,28,28])

# def recognition_network(images,reuse = False):
#     if reuse:
#             tf.get_variable_scope().reuse_variables()
#     output = tf.nn.relu(linear(images, 1000, scope='r_layer1', stddev=0.01))
#     output = tf.nn.softmax(linear(output, 10, scope='r_layer2', stddev=0.01))
#     return output
# # Define loss and optimizer
# y_clf = recognition_network(x)
# y_clf_g = recognition_network(out_vecter,reuse =True)

# cross_entropy_recog = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf, y_))
# cross_entropy_gen = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf_g, y_)) 
# # + \
# # tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_clf_g, y_))
# reconstruction_loss = tf.reduce_mean(tf.squared_difference(out_vecter,x))
# loss = reconstruction_loss + 7.2*cross_entropy_gen
# #cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
# #train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
# #t_vars = tf.trainable_variables()
# t_vars = tf.trainable_variables()
# contr_params = [var for var in t_vars if 'c_' in var.name]
# recog_params = [var for var in t_vars if 'r_' in var.name]

# optim = tf.train.AdamOptimizer(learning_rate=1e-3). \
#         minimize(loss,var_list=contr_params) 
# optim_recog = tf.train.AdamOptimizer(learning_rate=1e-3). \
#         minimize(cross_entropy_recog,var_list=recog_params) 

# session_conf = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7),
#                               allow_soft_placement=True,
#                               log_device_placement=False)
# sess = tf.InteractiveSession(config=session_conf)
# # Train
# tf.initialize_all_variables().run()
# batch_size = 100
# #pretrain recognition network
# # for step in range(5001):
# #     batch_xs, batch_ys = mnist.train.next_batch(batch_size)
# #     _ = sess.run(optim_recog,feed_dict={x: batch_xs,y_:batch_ys})
# for step in range(3001):
#     batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#     norm_image = np.random.uniform(0, 1, [batch_size,784]).astype(np.float32)
#     if np.random.rand(1)>1:
#         batch_xs = norm_image
#     x_bat = np.reshape(batch_xs,[-1,28,28])
#     samples,_,_,loss_,cross_entropy_gen_,reconstruction_loss_ = sess.run([out_image,
#                                                                         optim_recog,
#                                                                           optim,
#                                                                           loss,
#                                                                           cross_entropy_gen,
#                                                                           reconstruction_loss]
#                                                                           ,feed_dict={x: batch_xs,y_:batch_ys})
    
#     if step %50 ==0:
#         print('step : '+str(step)+' loss : '+ str(loss_)\
#               +' loss_reconstruction : '+ str(reconstruction_loss_)\
#               +' loss_gen : '+ str(cross_entropy_gen_))
   
#     if step % 2000 ==0:
#         scipy.misc.imsave('./sample/%s_input.png'% (step),x_bat[1])    
#         scipy.misc.imsave('./sample/%s_output.png'% (step),samples[1])
# batch_xs, batch_ys = mnist.test.next_batch(30)
# test_y = [[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
#           [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
#           [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
#           [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
#           [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
#           [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
#           [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
#           [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
#           [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
#           [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]]
# # feed_dict={x: batch_xs[:len(test_y)],y_:test_y}
# # samples_test= sess.run(out_image, feed_dict=feed_dict)
# # in_test= np.reshape(batch_xs[0:len(test_y)],[-1,28,28])
# # for i in range(len(test_y)):
# #     scipy.misc.imsave('./sample/%s_test_output.png'% (i),samples_test[i])
# #     scipy.misc.imsave('./sample/%s_test_input.png'% (i),in_test[i])